In [ ]:
# !pip install crewai exa_py


from crewai import Agent, Task, Crew, LLM
from tavily import TavilyClient
import os

In [ ]:
# Configure Exa API
llm = LLM(
    model="gemini/gemini-2.0-flash",
    temperature=0.7,
    api_key = ''
)


In [ ]:
# Utility function to demonstrate web search capability
from crewai.tools import tool

@tool("Internet  Tool")
def search_internet(query: str) -> str:
    """
    Search the web using Tavily API

    Args:
        query (str): The search query

    Returns:
        str: Search results from Tavily
    """
    client = TavilyClient(api_key="")

    try:
        response = client.search(query=query)
        return str(response)
    except Exception as e:
        return {"error": str(e)}


In [ ]:
# Define the Researcher Agent
researcher = Agent(
    role="Professional Online Researcher",
    goal="Search for recent and credible information on the latest gpt4.5, organize it, and provide source citations.",
    backstory="You are an expert researcher with access to advanced search tools. Use search internet tool by passing a string query",
    tools=[search_internet],  #search function as a tool
    verbose=True,
    llm=llm
)

In [ ]:
# Define the Writer Agent
writer = Agent(
    role="Professional Content Writer",
    goal="Transform research findings into well-structured, engaging, and accurate content.",
    backstory="You are a skilled writer who specializes in creating high-quality articles, reports, and summaries based on research data.",
    verbose=True,
    llm=llm
)


In [ ]:
# Define the Translator Agent
translator = Agent(
    role="Professional Translator",
    goal="Accurately translate content from English into the target language while preserving meaning, tone, and structure.",
    backstory="You are a multilingual translator with expertise in adapting content for different cultures and audiences.",
    verbose=True,
    llm=llm
)


In [ ]:
# Define Tasks for Each Agent
research_task = Task(
    description="Research the following topic: latest gpt4.5. Provide key facts, figures, and credible sources.",
    agent=researcher,
    expected_output="A research report. Provide string as input to the internet search tool"
)

In [ ]:
writing_task = Task(
    description="Write a well-structured article based on the research findings. Include an engaging introduction, clear sections, and proper citations.",
    agent=writer,
    context=[research_task],  # Depends on the research task
    expected_output="A detailed newsletter"
)

In [ ]:
translation_task = Task(
    description="Translate the written article into Spanish. Ensure the translation maintains the original meaning, tone, and structure.",
    agent=translator,
    context=[writing_task],  # Depends on the writing task
    expected_output="Translated newsletter"
)


In [ ]:
# Create the Crew
crew = Crew(
    agents=[researcher, writer, translator],
    tasks=[research_task, writing_task, translation_task]
)

In [ ]:
# Execute the Crew
results = crew.kickoff()

In [ ]:


# print research findings
print(results.tasks_output[0])

# print article
print(results.tasks_output[1])

# print translated article
print(results.tasks_output[2])

